In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

In [2]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [3]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
# choose a number of time steps
n_steps = 3
n_features = 1
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90
[70 80 90] 100


In [17]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv1d(in_channels=1,out_channels=3, kernel_size=1) # Espera de input, [n_batch,n_channel,n_size], solo importa el n_channel y n_size
        self.conv2 = nn.Conv1d(in_channels=3,out_channels=6, kernel_size=1)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(18,50)  # 5*5 from image dimension
        self.fc2 = nn.Linear(50,1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net().double()
print(net)

Net(
  (conv1): Conv1d(1, 3, kernel_size=(1,), stride=(1,))
  (conv2): Conv1d(3, 6, kernel_size=(1,), stride=(1,))
  (fc1): Linear(in_features=18, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=1, bias=True)
)


In [18]:
torch.randn(2, 1, 3 )

tensor([[[ 0.3116, -1.7767,  0.0327]],

        [[ 0.9528,  0.8918,  1.7453]]])

In [19]:
# Crear batch con tensores
    
#input = torch.from_numpy(np.array(X,dtype=float).reshape((X.shape[0],1, X.shape[1]))).double()
#input

In [20]:
#params = list(net.parameters())

In [21]:
#out = net.forward(input)
#out

In [22]:
#net.zero_grad()
#out.backward(torch.randn(6, 1))

In [23]:
#output = net(input)
#target = torch.Tensor(y).double()  # a dummy target, for example
#target = target.view(-1, 1)  # make it the same shape as output
#criterion = nn.MSELoss()
#target

In [24]:
#loss = criterion(output, target)
#print(loss)

In [25]:
#print(loss.grad_fn)  # MSELoss
#print(loss.grad_fn.next_functions[0][0])  # Linear
#print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [26]:
#net.zero_grad()     # zeroes the gradient buffers of all parameters

#print('conv1.bias.grad before backward')
#print(net.conv1.bias.grad)

#loss.backward()

#print('conv1.bias.grad after backward')
#print(net.conv1.bias.grad)

In [32]:
X

array([[10, 20, 30],
       [20, 30, 40],
       [30, 40, 50],
       [40, 50, 60],
       [50, 60, 70],
       [60, 70, 80],
       [70, 80, 90]])

In [38]:
## ENTRENEMOS LA RED
n_epoc = 10000
n_bacht = len(X)

# Crear los batches
tam_batch = int(len(X)/n_bacht)
bacht_input = []
for i in range(0,n_bacht):
    bacht_input.append([X[i*tam_batch:(i+1)*tam_batch]])


bacht_input = torch.from_numpy(np.array(bacht_input)).double()
bacht_target = []
for i in range(0,n_bacht):
    bacht_target.append([y[i*tam_batch:(i+1)*tam_batch]])

bacht_target = torch.from_numpy(np.array(bacht_target)).double()

# Funcion de pérdida
criterion = nn.MSELoss()

    
# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
for i in range(0,n_epoc):
    for j in range(0,n_bacht):
        optimizer.zero_grad()   # zero the gradient buffers
        output = net(bacht_input[j])
        loss = criterion(output, bacht_target[j])
        loss.backward()
        optimizer.step()    # Does the update

In [39]:
test = [80 , 90, 100]
prediccion = [110]

tensor_test = torch.from_numpy(np.array([[test]])).double()
print(tensor_test)
tensor_prediccion = torch.from_numpy(np.array(prediccion)).double()
tensor_prediccion

tensor([[[ 80.,  90., 100.]]], dtype=torch.float64)


tensor([110.], dtype=torch.float64)

In [40]:
net(tensor_test)

tensor([[70.8078]], dtype=torch.float64, grad_fn=<AddmmBackward0>)